In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
requirement_path = "/content/drive/MyDrive/stealthMode/Task-2/requirements.txt"
!pip install -r {requirement_path}

In [1]:
import cv2
import numpy as np
import csv
from ultralytics import YOLO

# Load YOLO model
model = YOLO("/content/drive/MyDrive/stealthMode/Task-2/best.pt")

# Set up video input and output
video_path = "/content/drive/MyDrive/stealthMode/AssignmentMaterial/15sec_input_720p.mp4"
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("/content/drive/MyDrive/stealthMode/output/player_tracking_output.avi",
                      cv2.VideoWriter_fourcc(*"XVID"), fps, (width, height))

# CSV setup
csv_file = open("/content/drive/MyDrive/stealthMode/output/player_tracking_output.csv", mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["frame", "id", "cx", "cy", "height", "jersey_color"])

# Track history
track_db = {}
next_id = 1
frame_num = 0
fps_target = 50
frame_interval = int(fps / fps_target) or 1
circle_radius = 25  # radius of visual tracking circle

# Dominant HSV hue from player crop
def get_dominant_color(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0], None, [180], [0, 180])
    return int(np.argmax(hist))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_num += 1
    if frame_num % frame_interval != 0:
        continue

    results = model(frame)[0]
    detections = results.boxes.xyxy.cpu().numpy() if results.boxes is not None else []

    centroids_drawn = []
    current_ids = []

    for box in detections:
        x1, y1, x2, y2 = map(int, box[:4])
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        h = y2 - y1

        # Skip if crop is invalid
        player_crop = frame[y1:y2, x1:x2]
        if player_crop.size == 0:
            continue

        jersey_color = get_dominant_color(player_crop)

        # Match with existing tracks
        matched_id = None
        min_dist = 50
        color_thresh = 15
        height_thresh = 0.2

        for tid, tdata in track_db.items():
            px, py = tdata["centroid"]
            ph = tdata["height"]
            pd_color = tdata["color"]

            dist = np.linalg.norm([cx - px, cy - py])
            color_diff = abs(jersey_color - pd_color)
            height_diff = abs(h - ph) / max(h, ph)

            if dist < min_dist and color_diff <= color_thresh and height_diff < height_thresh:
                matched_id = tid
                break

        if matched_id is None:
            matched_id = next_id
            next_id += 1

        # Update tracker
        track_db[matched_id] = {
            "centroid": (cx, cy),
            "color": jersey_color,
            "height": h,
            "last_frame": frame_num
        }
        current_ids.append(matched_id)

        # Write tracking data to CSV
        csv_writer.writerow([frame_num, matched_id, cx, cy, h, jersey_color])

        # Avoid overlapping circles
        overlap = False
        for (ox, oy) in centroids_drawn:
            if np.linalg.norm([cx - ox, cy - oy]) < circle_radius * 2:
                overlap = True
                break
        if overlap:
            continue  # skip drawing to avoid visual conflict

        centroids_drawn.append((cx, cy))

        # Draw circle at centroid
        cv2.circle(frame, (cx, cy), circle_radius, (0, 255, 0), 2)
        cv2.putText(frame, f"ID {matched_id}", (cx - 20, cy - circle_radius - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Remove disappeared players
    track_db = {
        tid: tdata for tid, tdata in track_db.items()
        if frame_num - tdata["last_frame"] <= int(fps * 2)
    }

    out.write(frame)

# Release resources
cap.release()
out.release()
csv_file.close()
print("Video with centroid-based tracking (non-overlapping) saved.")
print("CSV file with tracking data saved frame-wise.")



0: 384x640 1 ball, 16 players, 2 referees, 3066.1ms
Speed: 17.4ms preprocess, 3066.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 2366.1ms
Speed: 3.8ms preprocess, 2366.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2337.1ms
Speed: 2.9ms preprocess, 2337.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2397.6ms
Speed: 3.1ms preprocess, 2397.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2997.4ms
Speed: 4.2ms preprocess, 2997.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 3282.7ms
Speed: 3.5ms preprocess, 3282.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 2327.0ms
Speed: 3.6ms preprocess, 2327.0ms inference, 0.9ms